# Semi-discrete methods optimal transport and application to computational fluid dynamics

## Semi-discrete optimal transport

### Presentation of the toolbox

$\newcommand{\Lag}{\mathrm{Lag}}
\newcommand{\Ker}{\mathrm{Ker}}
\newcommand{\abs}[1]{\left| #1 \right|}
\newcommand{\sca}[2]{\langle#1|#2\rangle}
\newcommand{\Class}{\mathcal{C}}
\newcommand{\D}{\mathrm{D}}
\newcommand{\one}{\textbf{1}}
\newcommand{\hdots}{\dots}
\newcommand{\dd}{\mathrm{d}}
\newcommand{\Rsp}{\mathbb{R}}
\newcommand{\eps}{\varepsilon}
\newcommand{\nr}[1]{\|#1\|}
$
We study here the optimal transport problem between a constant density
$\rho=\one_P$ supported on a convex domain $\Omega$ of $\Rsp^2$ and a finitely supported
measure $\nu = \sum_{i\in I} \nu_i\delta_{y_i}$, where $I =
\{1,\hdots,N\}$. This problem can be recast as finding a Kantorovich
potential (a vector $\psi\in \Rsp^N$) satisfying the non-linear system
of equations

$$ \tag{P}\qquad \forall i\in I, G_i(\psi) = \nu_i, $$

where

$$
\begin{aligned}
  \label{eq:G}
  &G_i(\psi) = \rho(\Lag_i(\psi)) := \int_{\Lag_i(\psi)} \rho(x) \dd x,
  \\ &\Lag_i(\psi) = \{x \in \Omega\mid \forall j\in I, c(x,y_i) + \psi_i
  \leq c(x,y_j) + \psi_j \}.
\end{aligned}
$$

By default, we consider the cost $c(x,y) = \nr{x - y}^2$.  When we the set $Y$ is ambiguous, or
we want to consider another cost $c$, we will sometime use the heavier
notation $\Lag^c_i(y_1,\hdots,y_N;\psi_1,\hdots,\psi_N)$ for denoting
the $i$th Laguerre cell.  The set $\Lag_i(\psi) \subseteq P$ is called
the Laguerre cell of the $i$th point, and the collection
$(\Lag_i(\psi))_{1\leq i\leq N}$ is called the Laguerre
diagram.  We provide a simple wrapper around our code PySDOT to
easily perform computations on Laguerre diagrams (compute areas and
barycenters of Laguerre cells) when $\rho$ is constant. 

In [1]:
# imports
from pysdot.domain_types import ConvexPolyhedraAssembly
from pysdot import PowerDiagram
import numpy as np
from scipy.sparse.linalg import spsolve
from scipy.sparse import csr_matrix


# constructs a square domain, to be passed to the laguerre_* functions
def make_square(box=[0,0,1,1]):
    domain = ConvexPolyhedraAssembly()
    domain.add_box([box[0], box[1]], [box[2], box[3]])
    return domain

# construct a Laguerre diagram associated to a domain,
# a family of points Y (a table with N rows and 2 columns), and a set of potentials psi
def make_laguerre(domain, Y, psi):
    pd = PowerDiagram(domain)
    pd.set_positions(Y)
    pd.set_weights(-psi)
    return pd

# displays the Laguerre diagram
# optional arguments: disp_positions=True/False (display the points Y),
#                     disp_ids=True/False (show the number of each cell),
#                     disp_centroids=True/False (show the barycenter of each cell)
def laguerre_draw(domain, Y, psi, disp_ids=False, disp_positions=True, disp_centroids=False):
    return make_laguerre(domain, Y, psi).display_jupyter(disp_ids=disp_ids,
                                                         disp_positions=disp_positions,
                                                         disp_centroids=disp_centroids)

# computes the areas of the Laguerre cells intersected with the domain, and returns it as an array
# if der = True, also returns a sparse matrix representing the Jacobian of the areas with respect to psi
def laguerre_areas(domain, Y, psi, der = False):
    print(domain)
    if der:
        N = len(psi)
        mvs = make_laguerre(domain,Y,psi).der_integrals_wrt_weights()
        if mvs.error:
            raise ValueError("une des cellules de Laguerre est vide")
        return mvs.v_values, -csr_matrix((mvs.m_values, mvs.m_columns, mvs.m_offsets), shape=(N,N))
    else:
        return make_laguerre(domain,Y,psi).integrals()

# computes the barycenters of the Laguerre cells intersected with the domain, and returns it as a Nx2 array
def laguerre_barycenters(domain, Y, psi):
    return make_laguerre(domain, Y, psi).centroids()

### Numerical resolution of semi-discrete optimal transport

Equation (P) can be written as $G(\psi) = \nu$, where
$G(\psi) = (G_1(\psi),\hdots,G_N(\psi))\in\Rsp^N$ and where $ \nu =
(\nu_1,\hdots,\nu_N)\in\Rsp^N$.


**Q1.** The applet below allows to change the value of $\psi$ interactively. Try to solve the case $N=5$ by hand to get an understanding of what happens. 

In [2]:
# case N=5
from ipywidgets import interact, FloatSlider

N = 5
domain = make_square()
Y = np.array([[.1,.1],[.7,.2], [.4,.8], [.3,.5], [.7,.7]])
    
def disp(**kwargs):
    psi = np.array([p for p in kwargs.values()])
    print("areas=", laguerre_areas(domain, Y, psi))
    return laguerre_draw(domain, Y, psi, disp_positions=True, disp_ids=True, disp_centroids=False)
    
interact(disp, **{"psi%i"%i : FloatSlider(min=-.5,max=.5,step=0.005,value=0.0) for i in range(N)});

interactive(children=(FloatSlider(value=0.0, description='psi0', max=0.5, min=-0.5, step=0.005), FloatSlider(v…

**Q2.** Start with $N \in \{5,10,20\}$ random points in the square $[0,\frac{1}{2}]^2$, $\Omega = [0,1]^2$, and $\nu = (\frac{1}{N},\hdots,\frac{1}{N})\in\Rsp^N$. Then try to use the function `scipy.optimize.fsolve`  to solve equation (P).

In [3]:
from scipy.optimize import fsolve

N = 10
domain = make_square()
Y = .5*np.random.rand(N, 2)
nu = np.ones(N) / N
G = lambda psi: laguerre_areas(domain, Y, psi)

psi = fsolve(lambda psi: G(psi) - nu, np.zeros(N))
print(np.linalg.norm(G(psi) - nu))

1.1855457883870977e-11


**Q3.** Prove or check numerically that $G(\psi+e) = G(\psi)$, where $e =
(1,\hdots,1)\in\Rsp^N$ (the Laguerre cells are invariant by addition
  of a constant to each coordinates of $\psi$), and that $e \in \Ker
  \D G(\psi)$, so that $\D G(\psi)$ is never invertible.

In [8]:
N = 10
domain = make_square()
Y = .5*np.random.rand(N, 2)
G = lambda psi: laguerre_areas(domain, Y, psi)
DG = lambda psi: laguerre_areas(domain, Y, psi, der=True)[1]

psi = .001*np.random.rand(N)
e = np.ones(N)

print("|G(psi+e)-G(psi)| = %g" % np.linalg.norm(G(psi+e) - G(psi)))
print("|DG(psi)e| = %g" % np.linalg.norm(DG(psi)@e))

print(np.linalg.eigh(DG(psi).todense())[0])

|G(psi+e)-G(psi)| = 5.78748e-16
|DG(psi)e| = 2.07704e-16
[-6.42266021e+00 -5.88865033e+00 -5.32571005e+00 -3.25317924e+00
 -2.70205904e+00 -1.95826266e+00 -1.79973510e+00 -7.23195907e-01
 -4.53274575e-01 -5.53608452e-16]


We now turn to a much more efficient Newton's method for solving (P). Since $\D
G(\psi)$ is not invertible because of the invariance by addition of a
constant, we will fix $\psi_N = 0$. In practice, we simply remove the
last unknown ($\psi_N$) and also the last equation of the system
(which is then redundant, as $\sum_i G_i(\psi) = 1$), and define

$$\begin{aligned} F: \Rsp^{N-1} &\to \Rsp^{N-1}
  \\ \psi' = (\psi'_1,\hdots,\psi'_{N-1}) &\mapsto
  (G_1(\psi'_1,\hdots,\psi'_{N-1},0),\hdots,G_{N-1}(\psi'_1,\hdots,\psi'_{N-1},0)).
\end{aligned}
$$

The iterates in Newton's algorithm are denoted with an exponent,
${\psi'}^{(0)},\hdots,{\psi'}^{(k)}\in\Rsp^{N-1}$.  Linearizing the equation
$F({\psi'}^{(k)} + d^{(k)}) = \nu'$ (where $\nu' = (\nu_1,\hdots,\nu_{N-1})$, we get

\begin{equation} \tag{N}
  \begin{cases}
  \D F({\psi'}^{(k)})\cdot d^{(k)} = \nu' - F(\psi^{(k)}) \\
  {\psi'}^{(k+1)} = {\psi'}^{(k)} + d^{(k)}
\end{cases}
\end{equation}


**Q4.** Compute $\D F(\psi_1,\hdots,\psi_{N-1})$ explicitly and prove that if $\Ker \D
  G(\psi_1,\hdots,\psi_{N-1},0) = \Rsp e$, then $\D F(\psi_1,\hdots,\psi_{N-1})$ is
  invertible. Implement Newton's method for solving (P). Test
  it for $N \in \{10,100,1000\}$ random iid points in $[0,1]^2$, $P =
  [0,1]^2$ and $\nu = \frac{1}{N} e \in \Rsp^N$.

In [9]:
# use spsolve to solve the linear system

N = 100
domain = make_square()
Y = np.random.rand(N, 2)
nu = np.ones(N)/N
F = lambda psip: laguerre_areas(domain, Y, np.hstack((psip,0)))[0:-1]
DF = lambda psip: laguerre_areas(domain, Y, np.hstack((psip,0)), der=True)[1][0:-1,0:-1]

psip = np.zeros(N-1)
nup = nu[0:-1]
it = 0

while True:
    display(laguerre_draw(domain, Y, np.hstack((psip,0))))
    print("it=%d: |F - nu| = %g" % (it,np.linalg.norm(F(psip) - nup)))
    it += 1
    if np.linalg.norm(nup - F(psip)) <= 1e-6:
        break
    d = spsolve(DF(psip), nup - F(psip))
    psip = psip + d

<IPython.core.display.Javascript object>

it=0: |F - nu| = 0.0486464


<IPython.core.display.Javascript object>

it=1: |F - nu| = 0.0143682


<IPython.core.display.Javascript object>

it=2: |F - nu| = 0.000899253


<IPython.core.display.Javascript object>

it=3: |F - nu| = 1.04252e-05


<IPython.core.display.Javascript object>

it=4: |F - nu| = 1.69437e-09


A sufficient condition for the invertibility of $\D F(\psi')$ is that $\psi'\in \mathcal{K}$ where

$$\mathcal{K} = \{ \psi'\in\Rsp^{N-1} \mid \forall i
\in\{1,\hdots, N-1\}, \quad F_i(\psi')  > 0 \}.
$$

In practice, this condition is ensured throught a simple
backtracking procedure: one takes ${\psi'}^{(k+1)} = {\psi'}^{(k)} +
t^{(k)} d^{(k)}$ where $t^{(k)} \in (0,1]$ is chosen small enough so
  that ${\psi'}^{(k+1)} \in \mathcal{K}$. This procedure is
  implemented in the function `optimal_transport(P,Y,nu,psi0)`, provided below. The last 
  parameter `psi0`$\in \Rsp^N$ is optional, default to $0\in \Rsp^N$, and should be chosen so that
  
  \begin{equation} \label{eq:adm-psi0}
    \forall i\in I, G_i(\psi^{(0)}) > 0.
  \end{equation}
  

In [32]:
def optimal_transport(domain, Y, nu, psi0 = None, verbose=False, maxerr=1e-6, maxiter=10):
    if psi0 is None:
        psi0 = np.zeros(len(nu))
        
    def F(psip):
        g,h = laguerre_areas(domain, Y, np.hstack((psip,0)), der=True)
        return g[0:-1], h[0:-1,0:-1]
    
    psip = psi0[0:-1] - psi0[-1]
    nup = nu[0:-1]
    it = 0
    
    g,h = F(psip)
    for it in range(maxiter):
        err = np.linalg.norm(nup - g)
        if err <= maxerr:
            break
        d = spsolve(h, nup - g)
        t = 1.
        psip0 = psip.copy()
        while True:
            psip = psip0 + t*d
            try:
                g,h = F(psip)
            except:
                t = t/2
                continue
            break
        print("it %d: |err| = %g, t=%g" % (it, err, t))
    return np.hstack((psip,0))

Below, we show an example of use of the `optimal_transport` function, to transport a discretized Gaussian distribution to the uniform measure over the unit square $[0,1]^2$:

In [34]:
N = 5000
domain = make_square([0,0,1.,1.])
Y = .5+.2*np.random.randn(N, 2)
Y = np.minimum(np.maximum(Y,.01),.99)
nu = np.ones(N)/N
psi = optimal_transport(domain, Y, nu, verbose=True)
laguerre_draw(domain, Y, psi)

it 0: |err| = 0.0273525, t=0.5
it 1: |err| = 0.0142031, t=1
it 2: |err| = 0.00310211, t=1
it 3: |err| = 0.000785534, t=1
it 4: |err| = 9.62499e-05, t=1
it 5: |err| = 4.18265e-06, t=1


<IPython.core.display.Javascript object>

### Application to  Euler's equations for incompressible fluids

Given a domain $\Omega \subseteq \Rsp^2$, an initial velocity field $u_0:
\Omega\to\Rsp$ and an initial configuration of points $Y(0) =
(y_1(0),\hdots,y_N(0))\in\Rsp^d$, we consider the following system of
ODE as an approximation of Euler's equations for incompressible
fluids:

\begin{equation}  \label{eq:Euler}
  \begin{cases}
    \ddot{y}_i(t) = \frac{1}{\eps} (b_i(y_1(t),\hdots,y_N(t)) - y_i(t)) \\
    \dot{y}_i(0) = u_0(Y_i(t)) \\
  \end{cases}
\end{equation}

where we  $b_i(y_1,\hdots,y_N)$ is defined as follows:
- first, compute $\psi\in\Rsp^N$ the Kantorovich potential in
    the optimal transport problem between $\rho\equiv 1$ on $P$ and
    $\nu = \frac{1}{N}\sum_{i\in I} \delta_{y_i}$.
- $b_i(y_1,\hdots,y_N)$ is then the barycenter of the Laguerre
    cell $\Lag_i(y_1,\hdots,y_N;\psi_1,\hdots,\psi_N)$.


In what follows, we fix $\Omega = [-\frac{1}{2}, \frac{1}{2}]^2$.

**Q5.**  Write a function `B=proj_lebesgue(domain, Y)`, which takes as input
  a $N\times 2$ matrix $Y$ whose rows are the points $y_1,\hdots,y_N$
  and returns a $N\times 2$ matrix whose rows are
  $(b_i(y_1,\hdots,y_N))_{1\leq i\leq N}$. Observe its result when
  $y_1,\hdots,y_N$ are random iid in $\Omega$ or $\frac{1}{2}\Omega$.

In [44]:
def proj_lebesgue(domain, Y):
    N = Y.shape[0]
    psi = optimal_transport(domain, Y, np.ones(N)/N)
    return laguerre_barycenters(domain, Y, psi)

N=10
domain = make_square([-.5,-.5,.5,.5])
print(domain)
Y = -.5+np.random.rand(N, 2)
print(Y)

psi = np.zeros(N)
print(laguerre_areas(domain, Y, psi))
laguerre_draw(domain, Y, psi)
#B = proj_lebesgue(domain, Y)

[-0.5, -0.5, 0.5, 0.5]
[[-0.0073306  -0.24500888]
 [ 0.08056101  0.33128521]
 [ 0.47737656 -0.14622324]
 [-0.05304998  0.09048931]
 [ 0.24970372 -0.12222947]
 [-0.10574311 -0.22076848]
 [ 0.19576902  0.43183596]
 [-0.38281201  0.46508397]
 [-0.13127325  0.24928216]
 [ 0.39722707 -0.13333612]]
[0.00000000e+00 6.46739263e-02 1.77069895e-01 2.68847796e-02
 2.57497190e-02 0.00000000e+00 6.79534082e-01 0.00000000e+00
 1.46610511e-05 2.60729374e-02]


<IPython.core.display.Javascript object>